In [ ]:
import re

import benchlingapi
import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import toml
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# SigW storage vectors

In [ ]:
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
lib_parts = reg[("LIB", "parts")]
part_types = reg[("LIB", "parts", "Part types")]

In [ ]:
plib_maps.clear_cache()

In [ ]:
plasmids_to_make = [f"pLIB{i}" for i in range(213, 219)]

In [ ]:
plasmid_commands = {name: plib_plasmids[name]["Command"] for name in plasmids_to_make}
plasmid_commands

In [ ]:
plasmid_maps = {
    name: cmd_semantics.eval_command(cmd, reg.get)["_seq"]
    for name, cmd in plasmid_commands.items()
}

In [ ]:
plib_maps.update(plasmid_maps)

In [ ]:
plib_maps.save()

# FileClient test

In [ ]:
ptest_maps = reg[("pTEST", "maps")]

In [ ]:
ptest_maps.keys()

In [ ]:
ptest_maps._remote_folders

In [ ]:
ptest_maps.content[("foo", "bar", "baz")] = "bazxxx2"
ptest_maps.content[("foo", "test")] = "testxxx2"

In [ ]:
ptest_maps.local

In [ ]:
ptest_maps.bytes[("foo", "bar", "baz")]

In [ ]:
ptest_maps.clear_cache()

In [ ]:
ptest_maps.save()